This is the notebook for generating the wifi-features for all 204 buildings. [Dataset](https://www.kaggle.com/samratthapa/training-wifi-features-for-all-204-buildings)

You can use it to pretrain your unified model.Each row includes the 100 bssids with strongest rssi, the xy coordinates, floor label. It also includes the difference between the timestamp of the wifi and the nearest waypoint in the 'td' column. 

I adopted code from this notebook.https://www.kaggle.com/devinanzelmo/wifi-features

In [ ]:
packages = [
    '../input/indoor-locationnavigation-2021/indoor-location-competition-20-master/indoor-location-competition-20-master'
]
import sys
for pth in packages:
    sys.path.append(pth)

In [ ]:
from io_f import read_data_file
from compute_f import *

In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import gc
import json 

In [ ]:
base_path = '../input/indoor-location-navigation/'

In [ ]:
from pathlib import Path

In [ ]:
ssubm = pd.read_csv('../input/indoor-location-navigation/sample_submission.csv')


ssubm_df = ssubm["site_path_timestamp"].apply(lambda x: pd.Series(x.split("_")))
test_buildings = sorted(ssubm_df[0].value_counts().index.tolist())
used_buildings = sorted(os.listdir('../input/indoor-location-navigation/train'))

In [ ]:
len(test_buildings),len(used_buildings)

In [ ]:
column_reor = [x for x in range(0,200,2)]
column_reor.extend([x for x in range(1,200,2)])
column_reor.extend([200,201,202,203,204,205])

column_names = [f'bssid{x}' for x in range(100)]
column_names.extend([f'rssi{x}' for x in range(100)])
column_names.extend(['x','y','f','path','td','timestamps'])

In [ ]:
# get only the wifi bssid that occur over 1000 times(this number can be experimented with)
# these will be the only ones used when constructing features
uniques = np.array(["none"])
for building in test_buildings:
    folders = sorted(glob.glob(os.path.join(base_path,'train/'+building+'/*')))
    print(f'{building} :')
    dfs = list()
    for folder in folders:
        floor = folder.split('/')[-1]
        files = glob.glob(os.path.join(folder, "*.txt"))
        print(f'\t{floor}')
        for file in files:
            path_name = file.split('/')[-1].split('.')[0]
            temp_data = read_data_file(file)
            if len(temp_data.wifi)<50 or len(temp_data.waypoint)<2:
                continue
            wifi = temp_data.wifi
            waypoint = temp_data.waypoint
            
            df_wifi = pd.DataFrame(wifi)
            ext = []
            tar = []
            for gid, g in df_wifi.groupby(0):
                dists = list()
                for e, k in enumerate(waypoint):
                    dist = abs(int(gid) - int(k[0]))
                    dists.append(dist)
                nearest_wp_index = np.argmin(dists)
                
                g = g.drop_duplicates()
#                 Extracting the top 100 bssids with strongest signals.
                feat = g.sort_values(3).reset_index().iloc[:100,3:5].reindex(np.arange(100)).values.flatten()
                ext.append(feat)
                tar.append(np.array([float(waypoint[nearest_wp_index][1]), 
                                     float(waypoint[nearest_wp_index][2]),floor,path_name,np.min(dists),int(gid)]))
            data =  pd.DataFrame(np.hstack([ext,tar]))[column_reor]
            data.columns = column_names
            data.timestamps = data.timestamps.astype('int')
            data = data.sort_values(['timestamps'])
            data['t_diff'] = data['timestamps']-np.hstack([data['timestamps'][0],data['timestamps'][:-1].to_numpy()])
            dfs.append(data)
        gc.collect()
    dfs = pd.concat(dfs,0) 
    dfs.iloc[:,:100] = dfs.iloc[:,:100].fillna('none')
    dfs.iloc[:,100:200] = dfs.iloc[:,100:200].fillna(-999).astype('int32')
    uniques = np.concatenate([dfs.iloc[:,:100].stack().unique(),uniques])
    dfs.to_parquet(os.path.join(f'{building}_train.parquet'))
    print("Current unique bssid length: ",len(uniques))
    print('Saved File.........',Path(f'{building}_train.parquet').stat().st_size/1024/1024,"MB\n-------------------------------")

In [ ]:
train_files = glob.glob(os.path.join('', "*.parquet"))
li =[]
for filename in train_files:
    df = pd.read_parquet(filename)
    li.append(df)

train_all = pd.concat(li, axis=0, ignore_index=True)
train_all.to_parquet("train_all.parq")

In [ ]:
column_reor = [x for x in range(0,200,2)]
column_reor.extend([x for x in range(1,200,2)])
column_reor.extend([200,201])

column_names = [f'bssid{x}' for x in range(100)]
column_names.extend([f'rssi{x}' for x in range(100)])
column_names.extend(['path','timestamps'])

In [ ]:
unique_test = np.array(["none"])
dfs = list()
for path_name,g1 in ssubm_df.groupby(1):
    temp_data = read_data_file(os.path.join(base_path,'test/'+f'/{path_name}.txt'))
    wifi = temp_data.wifi
    
    df_wifi = pd.DataFrame(wifi)
    ext = []
    tar = []
    
    for gid, g in df_wifi.groupby(0):

        g = g.drop_duplicates()
        
        feat = g.sort_values(3).reset_index().iloc[:100,3:5].reindex(np.arange(100)).values.flatten()
        ext.append(feat)
        tar.append(np.array([path_name,int(gid)]))
    data =  pd.DataFrame(np.hstack([ext,tar]))[column_reor]
    data.columns = column_names
    data.timestamps = data.timestamps.astype('int')
    data = data.sort_values(['timestamps'])
    
    for timepoint in g1.iloc[:,2].tolist():
            deltas = np.abs(data['timestamps'].values - int(timepoint))
            min_delta_idx = deltas.argmin()
            data.loc[min_delta_idx,"timestamps"] = int(timepoint)
    data['t_diff'] = data['timestamps']-np.hstack([data['timestamps'][0],data['timestamps'][:-1].to_numpy()])
    
    dfs.append(data)
gc.collect()
dfs = pd.concat(dfs,0) 
dfs.iloc[:,:100] = dfs.iloc[:,:100].fillna('none')
dfs.iloc[:,100:200] = dfs.iloc[:,100:200].fillna(-999).astype('int32')
unique_test = np.concatenate([dfs.iloc[:,:100].stack().unique(),unique_test])
dfs.to_parquet('test_all.parq')

In [ ]:
# uniques includes the 'none' tag from each building 
unique_bssids = pd.DataFrame(np.unique(unique_test))
unique_bssids.to_csv('unique_bssids_test.csv',index=False)

In [ ]:
# uniques includes the 'none' tag from each building 
unique_bssids = pd.DataFrame(np.unique(uniques))
unique_bssids.to_csv('unique_bssids_train.csv',index=False)